# Train End to End Gaze

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(224*224*3, 1024)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        return x
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
model2 = MLP()
model2.initialize_weights()
# Load a pre-trained ResNet-18 model and modify the final layer
model = models.resnet18(weights="ResNet18_Weights.DEFAULT")
model.fc = nn.Linear(model.fc.in_features, 4)
#model=model2
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.00001)
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
import os
import torch
from tqdm import tqdm

# Training loop
num_epochs = 10  # Define your desired number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print("start epoch :"+str(epoch))
    # Add a progress bar for the training loop

    for images, gazes in tqdm(train_loader): # train_progress_bar:
        images = images.to(device)  # Convert one-hot to class index
        gazes=torch.argmax(gazes, dim=1).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, gazes)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Update the progress bar


    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

    # Save checkpoint
    if (epoch + 1) % save_epochs == 0:
        torch.save(model.state_dict(), os.path.join(ckpt_dir, f"epoch_{epoch+1}.ckpt"))

    # Validation loop
    if (epoch + 1) % validate_epochs == 0:
        model.eval()
        correct = 0
        total = 0
        # Add a progress bar for the validation loop
        valid_progress_bar = tqdm(valid_loader, desc='Validating', leave=False)
        with torch.no_grad():
            for images, gazes in valid_progress_bar:
                images, gazes = images.to(device), torch.argmax(gazes, dim=1).to(device)  # Convert one-hot to class index

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += gazes.size(0)
                correct += (predicted == gazes).sum().item()

        print(f"Validation Accuracy: {100 * correct / total}%")

print("Training completed.")


# Train Depth+Img Detection

In [85]:
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

batch=32
data_path = r"D:\Datasets\Talis_frames15_v2"  # Provide your data path here

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
validate_epochs = 1  # Example: validate every epoch
save_epochs = 2     # Example: save the model every 2 epochs
ckpt_dir = r"D:\Datasets\Talis_frames15_v2\checkpoints"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip the image horizontally
    transforms.RandomVerticalFlip(p=0.5),  # Randomly flip the image vertically
    transforms.RandomRotation(degrees=15),  # Randomly rotate the image by up to 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly change the brightness, contrast, saturation, and hue
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.75, 1.33)),  # Randomly crop and resize the image
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
class GazeDataset(Dataset):
    def __init__(self, csv_file, data_path, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file.
            data_path (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.data_path = data_path
        self.transform = transform
        # Create a dictionary mapping each unique gaze value to a unique integer
        self.gaze_to_int = {gaze: idx for idx, gaze in enumerate(self.data_frame['gaze'].unique())}
        self.num_classes = len(self.gaze_to_int)
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_path, self.data_frame['imgID'][idx])  # Assuming imgID is in the first column
        image = Image.open(img_name)
        dep_name = os.path.join(self.data_path+"_depth", self.data_frame['imgID'][idx])
        depth = Image.open(img_name)

        if self.transform:
            image = self.transform(image)
            depth = transform2(depth)
        gaze = self.data_frame['gaze'][idx]   # Assuming gaze is in the second column
        gaze_idx = self.gaze_to_int[gaze]

        # Convert gaze_idx to one-hot encoded vector
        one_hot_gaze = torch.zeros(self.num_classes)
        one_hot_gaze[gaze_idx] = 1
        return image, depth, one_hot_gaze

# Create datasets
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming that your CSV file has two columns: 'imgID' for image file names and 'gaze' for gaze labels
data_frame = pd.read_csv('D:\Datasets\Talis_frames15_v2\labels_and_features_TRAIN.csv')

# Count the frequency of each class
class_sample_counts = data_frame['gaze'].value_counts().sort_index().to_numpy()
# Compute weights for each class
weights = 1.0 / class_sample_counts
# Create a weight for each sample in the dataset
sample_weights = weights[data_frame['gaze'].replace({gaze: idx for idx, gaze in enumerate(data_frame['gaze'].unique())}).to_numpy()]
from torch.utils.data import WeightedRandomSampler, DataLoader
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)



train_dataset = GazeDataset(csv_file='D:\Datasets\Talis_frames15_v2\labels_and_features_TRAIN.csv', data_path=data_path, transform=transform)
valid_dataset = GazeDataset(csv_file='D:\Datasets\Talis_frames15_v2\labels_and_features_VAL.csv', data_path=data_path, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch, sampler=sampler, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=batch, shuffle=False, num_workers=0)

# Load a pre-trained ResNet-18 model and modify the final layer
model = models.resnet18(weights="ResNet18_Weights.DEFAULT")
model.fc = nn.Linear(model.fc.in_features, 4)
model.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

#model=model2
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.00001)
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
import os
import torch
from tqdm import tqdm

# Training loop
num_epochs = 10  # Define your desired number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print("start epoch :"+str(epoch))
    # Add a progress bar for the training loop

    for images,depth, gazes in tqdm(train_loader): # train_progress_bar:
        inputs = torch.cat([images, depth], dim=1)
        # print(inputs.shape)
        inputs, gazes = inputs.to(device), torch.argmax(gazes, dim=1).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, gazes)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Update the progress bar


    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

    # Save checkpoint
    if (epoch + 1) % save_epochs == 0:
        torch.save(model.state_dict(), os.path.join(ckpt_dir, f"epoch_{epoch+1}.ckpt"))

    # Validation loop
    if (epoch + 1) % validate_epochs == 0:
        model.eval()
        correct = 0
        total = 0
        # Add a progress bar for the validation loop
        valid_progress_bar = tqdm(valid_loader, desc='Validating', leave=False)
        with torch.no_grad():
            for images, depth, gazes in valid_progress_bar:
                inputs = torch.cat([images, depth], dim=1)
                inputs, gazes = inputs.to(device), torch.argmax(gazes, dim=1).to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += gazes.size(0)
                correct += (predicted == gazes).sum().item()

        print(f"Validation Accuracy: {100 * correct / total}%")

print("Training completed.")




start epoch :0


100%|██████████| 77/77 [01:23<00:00,  1.08s/it]


Epoch [1/10], Loss: 1.256945919680905


Validating:   5%|▌         | 1/19 [00:01<00:18,  1.03s/it]

torch.Size([32, 6, 224, 224])


...

Validating:  95%|█████████▍| 18/19 [00:18<00:01,  1.03s/it]

torch.Size([32, 6, 224, 224])


torch.Size([29, 6, 224, 224])
Validation Accuracy: 25.950413223140497%
start epoch :2


 61%|██████    | 47/77 [00:50<00:32,  1.07s/it]


KeyboardInterrupt: 

# Train Depth+Img+Head+Eye Detection

In [ ]:
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

batch=32
data_path = r"C:\Datasets\Engagement"  # Provide your data path here

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
validate_epochs = 1  # Example: validate every epoch
save_epochs = 2     # Example: save the model every 2 epochs
ckpt_dir = r"C:\Datasets\Engagement\checkpoints"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip the image horizontally
    #transforms.RandomVerticalFlip(p=0.5),  # Randomly flip the image vertically
    #transforms.RandomRotation(degrees=15),  # Randomly rotate the image by up to 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly change the brightness, contrast, saturation, and hue
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.75, 1.33)),  # Randomly crop and resize the image
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform3 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


transform2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
class GazeDataset(Dataset):
    def __init__(self, csv_file, data_path, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file.
            data_path (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.data_path = data_path
        self.transform = transform
        # Create a dictionary mapping each unique gaze value to a unique integer
        self.gaze_to_int ={3: 0, 2: 1, 1: 0, 4: 1}
        self.num_classes = 2
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_path+"\image_original", self.data_frame['imgID'][idx])  # Assuming imgID is in the first column
        image = Image.open(img_name)
        dep_name = os.path.join(self.data_path+"\depth_intermediate", self.data_frame['imgID'][idx])
        depth = Image.open(dep_name)
        gaze_name = os.path.join(self.data_path+"\predict_heatmap", self.data_frame['imgID'][idx])
        gaze_img = Image.open(gaze_name)

        if self.transform:
            image = self.transform(image)
            depth = transform2(depth)
            gaze_img = transform2(gaze_img)

        
        gaze = self.data_frame['gaze'][idx]   # Assuming gaze is in the second column
        gaze_idx = self.gaze_to_int[gaze]

        # Convert gaze_idx to one-hot encoded vector
        one_hot_gaze = torch.zeros(self.num_classes)
        one_hot_gaze[gaze_idx] = 1
        return image, depth,gaze_img, one_hot_gaze

# Create datasets
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming that your CSV file has two columns: 'imgID' for image file names and 'gaze' for gaze labels
data_frame = pd.read_csv('C:\Datasets\labels_and_features_TRAIN.csv')


train_dataset = GazeDataset(csv_file='C:\Datasets\labels_and_features_TRAIN.csv', data_path=data_path, transform=transform)
valid_dataset = GazeDataset(csv_file='C:\Datasets\labels_and_features_VAL.csv', data_path=data_path, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=batch, shuffle=False, num_workers=0)

# Load a pre-trained ResNet-18 model and modify the final layer
model = models.resnet18(weights="ResNet18_Weights.DEFAULT")
model.fc = nn.Linear(model.fc.in_features, 4)
model.conv1 = nn.Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

#model=model2
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.00001)
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
import os
import torch
from tqdm import tqdm

# Training loop
num_epochs = 10  # Define your desired number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print("start epoch :"+str(epoch))
    # Add a progress bar for the training loop

    for images,depth, gaze, gazes in tqdm(train_loader): # train_progress_bar:

        inputs = torch.cat([images, depth, gaze], dim=1)
        # print(inputs.shape)
        inputs, gazes = inputs.to(device), torch.argmax(gazes, dim=1).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, gazes)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Update the progress bar


    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

    # Save checkpoint
    if (epoch + 1) % save_epochs == 0:
        torch.save(model.state_dict(), os.path.join(ckpt_dir, f"epoch_{epoch+1}.ckpt"))

    # Validation loop
    if (epoch + 1) % validate_epochs == 0:
        model.eval()
        correct = 0
        total = 0
        # Add a progress bar for the validation loop
        valid_progress_bar = tqdm(valid_loader, desc='Validating', leave=False)
        with torch.no_grad():
            for images, depth, gaze, gazes in valid_progress_bar:
                inputs = torch.cat([images, depth, gaze ], dim=1)
                inputs, gazes = inputs.to(device), torch.argmax(gazes, dim=1).to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += gazes.size(0)
                correct += (predicted == gazes).sum().item()

        print(f"Validation Accuracy: {100 * correct / total}%")

print("Training completed.")




start epoch :0


100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [02:35<00:00,  2.02s/it]


Epoch [1/10], Loss: 1.341203099721438


Validation Accuracy: 47.27272727272727%
start epoch :1


100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [02:28<00:00,  1.93s/it]


Epoch [2/10], Loss: 1.2730936214521333


Validation Accuracy: 50.082644628099175%
start epoch :2


 13%|██████████▋                                                                       | 10/77 [00:19<02:08,  1.92s/it]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

# Function to plot images
def plot_images(image, depth, gaze, title):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(transforms.ToPILImage()(image))
    plt.title("Original Image")
    plt.subplot(1, 3, 2)
    plt.imshow(transforms.ToPILImage()(depth))
    plt.title("Depth Image")
    plt.subplot(1, 3, 3)
    plt.imshow(transforms.ToPILImage()(gaze))
    plt.title("Gaze Image")
    plt.suptitle(title)
    plt.show()


# Randomly pick 3 images from the validation dataset
random_indices = random.sample(range(len(valid_dataset)), 3)

for idx in random_indices:
    image, depth, gaze, label = valid_dataset[idx]
    input_tensor = torch.cat([image.unsqueeze(0), depth.unsqueeze(0), gaze.unsqueeze(0)], dim=1)
    input_tensor = input_tensor.to(device)
    
    # Get prediction
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output.data, 1)

    # Plotting
    plot_images(image, depth, gaze, f"Label: {label}, Prediction: {predicted.item()}")

In [12]:
train_dataset.gaze_to_int

{3: 0, 2: 1, 1: 2, 4: 3}

# Gaze Only

In [8]:
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

batch=32
data_path = r"C:\Datasets\Engagement"  # Provide your data path here

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
validate_epochs = 1  # Example: validate every epoch
save_epochs = 2     # Example: save the model every 2 epochs
ckpt_dir = r"C:\Datasets\Engagement\checkpoints"


transform2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    
])
class GazeDataset(Dataset):
    def __init__(self, csv_file, data_path, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file.
            data_path (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.data_path = data_path
        self.transform = transform
        # Create a dictionary mapping each unique gaze value to a unique integer
        self.gaze_to_int ={3: 0, 2: 1, 1: 0, 4: 1}
        self.num_classes = 2
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_path+"\image_original", self.data_frame['imgID'][idx])  # Assuming imgID is in the first column
        image = Image.open(img_name)
        dep_name = os.path.join(self.data_path+"\depth_intermediate", self.data_frame['imgID'][idx])
        depth = Image.open(img_name)
        gaze_name = os.path.join(self.data_path+"\predict_heatmap", self.data_frame['imgID'][idx])
        gaze_img = Image.open(gaze_name)

        if self.transform:
            image = self.transform(image)
            depth = transform2(depth)
            gaze_img = transform2(gaze_img)
        gaze = self.data_frame['gaze'][idx]   
        gaze_idx = self.gaze_to_int[gaze]

        # Convert gaze_idx to one-hot encoded vector
        one_hot_gaze = torch.zeros(self.num_classes)
        one_hot_gaze[gaze_idx] = 1
        return image, depth,gaze_img, one_hot_gaze

# Create datasets
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming that your CSV file has two columns: 'imgID' for image file names and 'gaze' for gaze labels
data_frame = pd.read_csv('C:\Datasets\labels_and_features_TRAIN.csv')

 

train_dataset = GazeDataset(csv_file='C:\Datasets\labels_and_features_TRAIN.csv', data_path=data_path, transform=transform)
valid_dataset = GazeDataset(csv_file='C:\Datasets\labels_and_features_VAL.csv', data_path=data_path, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=batch, shuffle=False, num_workers=0)

# Load a pre-trained ResNet-18 model and modify the final layer
model = models.resnet18(weights="ResNet18_Weights.DEFAULT")
model.fc = nn.Linear(model.fc.in_features, 2)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

#model=model2
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.00001)
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
import os
import torch
from tqdm import tqdm

# Training loop
num_epochs = 10  # Define your desired number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print("start epoch :"+str(epoch))
    # Add a progress bar for the training loop

    for images,depth, gaze, gazes in tqdm(train_loader): # train_progress_bar:

        inputs = torch.cat([gaze], dim=1)
        # print(inputs.shape)
        inputs, gazes = inputs.to(device), torch.argmax(gazes, dim=1).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, gazes)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Update the progress bar


    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

    # Save checkpoint
    if (epoch + 1) % save_epochs == 0:
        torch.save(model.state_dict(), os.path.join(ckpt_dir, f"epoch_{epoch+1}.ckpt"))

    # Validation loop
    if (epoch + 1) % validate_epochs == 0:
        model.eval()
        correct = 0
        total = 0
        # Add a progress bar for the validation loop
        valid_progress_bar = tqdm(valid_loader, desc='Validating', leave=False)
        with torch.no_grad():
            for images, depth, gaze, gazes in valid_progress_bar:
                inputs = torch.cat([gaze ], dim=1)
                inputs, gazes = inputs.to(device), torch.argmax(gazes, dim=1).to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += gazes.size(0)
                correct += (predicted == gazes).sum().item()

        print(f"Validation Accuracy: {100 * correct / total}%")

print("Training completed.")




start epoch :0


100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [02:48<00:00,  2.19s/it]


Epoch [1/10], Loss: 0.8281658988494378


Validation Accuracy: 50.247933884297524%
start epoch :1


100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [02:50<00:00,  2.21s/it]


...

Epoch [9/10], Loss: 0.6552678571118937


Validation Accuracy: 59.83471074380165%
start epoch :9


100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [03:18<00:00,  2.58s/it]


Epoch [10/10], Loss: 0.6499900825611957


Validation Accuracy: 59.33884297520661%
Training completed.


In [12]:
import os
import numpy as np


# Example usage:
filepath=r"C:\Datasets\Engagement\npy\000000.npy"
data = np.load(filepath)
print(f"Size of {data.shape}")

Size of (720, 1280)


In [13]:
data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [4, 4, 4, ..., 0, 0, 0],
       [4, 4, 4, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [15]:
np.unique(data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17], dtype=uint8)